## DNN Frontier 결과 분석

### modules import

In [1]:
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.utils import get_file, plot_model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

In [2]:
tf.__version__

'2.12.0'

### 데이터 로드

In [3]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
import os
print(os.getcwd())

/content


In [ ]:
os.chdir("/content/gdrive/MyDrive/DNN_Frontier")

In [ ]:
# read test_categorized df
# read trad_eff_crs
# read trad_eff_vrs

# change their eff as card range 0.1 0.05
# calculate the accuracy
# compare with DNN

# extract the row which trad eff is better than DNN

# find out the reason why DNN Frontier is worse than traditonal Frontier some times
#######################################

# change trad eff as card range 0.2 ,0.1, 0.05
# compare with DNN

l = [10000,20000,40000,80000,160000,320000,640000] # DMU to
g = [1280000] # DMU

h = [1,2,3,4,5] # Output
i = [1,2,3,4,5] # Input
j = [1,2] # Orientation
k = [1,2,3] # Card Range

#re = pd.DataFrame(columns = ['DMU', 'Output', 'Input', 'Orientation', 'Card_Range', 
#                             'CRS_acc', 'VRS_acc', 'DNN_acc','full_DNN_acc', 'acc_Applicable', 
#                             'CRS_RMSE', 'VRS_RMSE', 'DNN_RMSE', 'full_DNN_RMSE', 'rmse_Applicable', 
#                             'Applicable'])

re_df = pd.DataFrame(columns = ['DMU', 'Output', 'Input', 'Orientation', 'Card_Range', 
                             'CRS_acc', 'VRS_acc', 'DNN_acc', 'acc_Applicable', 
                             'CRS_RMSE', 'VRS_RMSE', 'DNN_RMSE', 'rmse_Applicable', 
                             'Applicable'])
m = 1 # 열 사이에 넣기 위한 도구

for f in l:
  re = pd.DataFrame(columns = ['128to{}_DNN_acc'.format(m), '128to{}_DNN_RMSE'.format(m)])
  # re = pd.concat([re_df,re], axis = 1)
  for a in g:
    for b in h:
      for c in i:
        for d in j:
          for e in k:
            
            if b + c < 4:
              continue


            r_1 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/comparison_result/result_comparison_{}.csv'.format(f) # comparison dataset
            r_2 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/trad_eff/{}_{}{}_{}_trad_eff.csv'.format(f,b,c,d) # CRS & VRS Result (not Categorized)
            r_3 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/re_DNN_Frontier_Classification/pred_df/other/Cat_{}to{}_{}{}_{}_{}_Pred_df.csv'.format(a,f,b,c,d,e) # Label Data & DNN Result (Categorized)

            df = pd.read_csv(r_3)
            df_2 = pd.read_csv(r_2)
            df = df.iloc[:,1:3]
            df_2 = df_2.iloc[:,1:3]

            # transform categorized_df
            if e == 1:
              bins = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
              lab = ['J','I','H','G','F','E','D','C','B','A']
            elif e == 2:
              bins = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
              lab = ['T','S','R','Q','P','O','N','M','L','K','J','I','H','G','F','E','D','C','B','A']
            else:
              bins = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
              lab = ['E','D','C','B','A']

            target_dat_1 = df_2.iloc[:,0]
            target_dat_2 = df_2.iloc[:,1]


            df_2['eff_lev_crs'] = pd.cut(target_dat_1, bins=bins, labels = lab)
            df_2['eff_lev_vrs'] = pd.cut(target_dat_2, bins=bins, labels = lab)        


            # target and transform trad_eff
            X = df_2.iloc[:, 2:4]

            LabE_1 = LabelEncoder()
            LabE_2 = LabelEncoder()
            
            Y_obj_1 = df_2['eff_lev_crs']
            Y_obj_2 = df_2['eff_lev_vrs']

            LabE_1.fit(Y_obj_1)
            Y_1 = LabE_1.transform(Y_obj_1)
            X['CRS_eff'] = Y_1

            LabE_2.fit(Y_obj_2)
            Y_2 = LabE_2.transform(Y_obj_2)
            X['VRS_eff'] = Y_2

            trad_eff_df = X.iloc[:,2:4]


            # transform label_range_to_singluar_data
            if e == 1:
              trad_eff_df['eff_crs_p'] = (2-(2*(trad_eff_df['CRS_eff'])+1)/10)/2
              trad_eff_df['eff_vrs_p'] = (2-(2*(trad_eff_df['VRS_eff'])+1)/10)/2
              
              df['eff_lev_p'] = (2-(2*(df['eff_lev'])+1)/10)/2
              df['pred_p'] = (2-(2*(df['pred'])+1)/10)/2

            elif e == 2:
              trad_eff_df['eff_crs_p'] = 1-(2*(trad_eff_df['CRS_eff'])+1)/40
              trad_eff_df['eff_vrs_p'] = 1-(2*(trad_eff_df['VRS_eff'])+1)/40

              df['eff_lev_p'] = 1-(2*(df['eff_lev'])+1)/40
              df['pred_p'] = 1-(2*(df['pred'])+1)/40

            else:
              trad_eff_df['eff_crs_p'] = 1-(2*(trad_eff_df['CRS_eff'])+1)/10
              trad_eff_df['eff_vrs_p'] = 1-(2*(trad_eff_df['VRS_eff'])+1)/10

              df['eff_lev_p'] = 1-(2*(df['eff_lev'])+1)/10
              df['pred_p'] = 1-(2*(df['pred'])+1)/10


  ################################################################################

            # calculate the accuracy
            df_dict = df.to_dict() # DNN Result
            dic_val = df_dict['eff_lev'].values()
            dict_list = list(dic_val)

            dnn_val = df_dict['pred'].values()
            dnn_list = list(dnn_val)


            trad_eff_df_dict = trad_eff_df.to_dict()
            dic_val_2 = trad_eff_df_dict['CRS_eff'].values()
            dict_list_2 = list(dic_val_2)

            dic_val_3 = trad_eff_df_dict['VRS_eff'].values()
            dict_list_3 = list(dic_val_3)

#            crs_acc = accuracy_score(dict_list, dict_list_2)
#            vrs_acc = accuracy_score(dict_list, dict_list_3)
            dnn_acc = accuracy_score(dict_list, dnn_list)

  ################################################################################

            # calculate RMSE
 #           crs_RMSE = mean_squared_error(df['eff_lev_p'], trad_eff_df['eff_crs_p'])**0.5
 #           vrs_RMSE = mean_squared_error(df['eff_lev_p'], trad_eff_df['eff_vrs_p'])**0.5
            dnn_RMSE = mean_squared_error(df['eff_lev_p'], df['pred_p'])**0.5

  ################################################################################

            ## Result
            n_re = pd.DataFrame(columns = ['128to{}_DNN_acc'.format(m), '128to{}_DNN_RMSE'.format(m)])
            n_re['128to{}_DNN_acc'.format(m)] = [dnn_acc]
            n_re['128to{}_DNN_RMSE'.format(m)] = [dnn_RMSE]

            re = pd.concat([re, n_re], axis = 0)

            re_df = pd.read_csv(r_1)
  m = m*2
  re.reset_index(drop = True, inplace = True)
  re.to_csv('/content/gdrive/MyDrive/DNN_Frontier/0902_data/comparison_result/comparison_1280000to{}.csv'.format(f), encoding = 'UTF-8')

In [ ]:
re

,DMU,Output,Input,Orientation,Card_Range,CRS_acc,VRS_acc,DNN_acc,acc_Applicable,CRS_RMSE,VRS_RMSE,DNN_RMSE,rmse_Applicable,Applicable,128to1_DNN_acc,128to1_DNN_RMSE


In [ ]:
re.to_csv('/content/gdrive/MyDrive/DNN_Frontier/0902_data/comparison_result/result_comparison_1280000.csv', encoding = 'UTF-8')

In [ ]:
######################################################################################################################################



```
# 코드로 형식 지정됨
```

### 완료 데이터 로드_2


In [ ]:
A = [10000, 20000, 40000, 80000, 160000, 320000, 640000]
for i in A:
  dt = pd.read_csv('/content/gdrive/MyDrive/DNN_Frontier/0902_data/comparison_result/result_comparison_{}.csv'.format(i))
  dt2 = pd.read_csv('/content/gdrive/MyDrive/DNN_Frontier/0902_data/comparison_result/comparison_1280000to{}.csv'.format(i))

  dataset = pd.concat([ dt.iloc[:,0:9], dt2.iloc[:,1], dt.iloc[:,9:13], dt2.iloc[:,2], dt.iloc[:,-2] ], axis = 1).reset_index(drop=True)
  dataset.to_csv('/content/gdrive/MyDrive/DNN_Frontier/0902_data/comparison_result/result_comparison_1280000to{}.csv'.format(i), encoding = 'UTF-8')

In [ ]:
dataset = pd.read_csv('/content/gdrive/MyDrive/DNN_Frontier/0902_data/comparison_result/result_comparison_1280000.csv')
A = [10000, 20000, 40000, 80000, 160000, 320000, 640000]
x = 0
for i in A:
  
  sub = pd.read_csv('/content/gdrive/MyDrive/DNN_Frontier/0902_data/comparison_result/result_comparison_1280000to{}.csv'.format(i)).iloc[:,1:]
  
  dataset = pd.concat([ dataset.iloc[:,0:9+x], sub.iloc[:,0], dataset.iloc[:,9+x:13+2*x], sub.iloc[:,1],dataset.iloc[:,-2:] ], axis = 1).reset_index(drop=True)
  
  x += 1

dataset.head()

,Unnamed: 0,DMU,Output,Input,Orientation,Card_Range,CRS_acc,VRS_acc,DNN_acc,128to1_DNN_acc,...,DNN_RMSE,128to1_DNN_RMSE,128to2_DNN_RMSE,128to4_DNN_RMSE,128to8_DNN_RMSE,128to16_DNN_RMSE,128to32_DNN_RMSE,128to64_DNN_RMSE,rmse_Applicable,Applicable
0,0,1280000,1,3,1,0.10,0.134948,0.116207,0.427964,0.249444,...,0.165950,0.148080,0.150508,0.142127,0.148392,0.145882,0.145654,0.145509,Y,Y
1,1,1280000,1,3,1,0.05,0.065412,0.065790,0.260278,0.135000,...,0.165395,0.159252,0.157114,0.150028,0.155195,0.151709,0.152511,0.152230,Y,Y
2,2,1280000,1,3,1,0.20,0.269744,0.221363,0.627027,0.489444,...,0.158875,0.174292,0.173622,0.167879,0.172264,0.170465,0.170182,0.169783,Y,Y
3,3,1280000,1,3,2,0.10,0.081089,0.099536,0.239379,0.217778,...,0.279737,0.266552,0.266656,0.261526,0.268386,0.268425,0.266893,0.268331,Y,Y
4,4,1280000,1,3,2,0.05,0.038303,0.058685,0.127166,0.105556,...,0.292515,0.272009,0.273218,0.268129,0.274478,0.273923,0.273259,0.274517,Y,Y


In [ ]:
dataset.to_csv('/content/gdrive/MyDrive/DNN_Frontier/0902_data/comparison_result/result_comparison_1280000toAll.csv', encoding = 'UTF-8')

In [ ]:
dataset = pd.read_csv('/content/gdrive/MyDrive/DNN_Frontier/0902_data/comparison_result/result_comparison_1280000.csv')
dataset.head()

,Unnamed: 0,DMU,Output,Input,Orientation,Card_Range,CRS_acc,VRS_acc,DNN_acc,acc_Applicable,CRS_RMSE,VRS_RMSE,DNN_RMSE,rmse_Applicable,Applicable
0,0,1280000,1,3,1,0.10,0.134948,0.116207,0.427964,Y,0.277131,0.352181,0.165950,Y,Y
1,1,1280000,1,3,1,0.05,0.065412,0.065790,0.260278,Y,0.271652,0.292295,0.165395,Y,Y
2,2,1280000,1,3,1,0.20,0.269744,0.221363,0.627027,Y,0.304813,0.363605,0.158875,Y,Y
3,3,1280000,1,3,2,0.10,0.081089,0.099536,0.239379,Y,0.417089,0.378799,0.279737,Y,Y
4,4,1280000,1,3,2,0.05,0.038303,0.058685,0.127166,Y,0.440280,0.339415,0.292515,Y,Y




```
# 코드로 형식 지정됨
```

### 완료 데이터 로드


In [ ]:
A = [10000, 20000, 40000, 80000, 160000, 320000, 640000,1280000]
for i in A:
  globals()["re_{}".format(i)] = pd.read_csv('/content/gdrive/MyDrive/DNN_Frontier/0902_data/comparison_result/result_comparison_{}.csv'.format(i)).iloc[:,1:]

In [ ]:
# A = [10000, 20000, 40000, 80000, 160000]
# for i in A:
#   globals()["crs_acc_{}_mean".format(i)] = globals()["re_{}".format(i)]['CRS_acc'].mean()
#   globals()["vrs_acc_{}_mean".format(i)] = globals()["re_{}".format(i)]['VRS_acc'].mean()
#   globals()["dnn_acc_{}_mean".format(i)] = globals()["re_{}".format(i)]['DNN_acc'].mean()
#   globals()["crs_rmse_{}_mean".format(i)] = globals()["re_{}".format(i)]['CRS_RMSE'].mean()
#   globals()["vrs_rmse_{}_mean".format(i)] = globals()["re_{}".format(i)]['VRS_RMSE'].mean()
#   globals()["dnn_rmse_{}_mean".format(i)] = globals()["re_{}".format(i)]['DNN_RMSE'].mean()

In [ ]:
A = [10000, 20000, 40000, 80000, 160000, 320000, 640000,1280000]
df = pd.DataFrame(columns = ['DMU', 'CRS_acc_mean', 'VRS_acc_mean', 'DNN_acc_mean','CRS_RMSE_mean','VRS_RMSE_mean','DNN_RMSE_mean'])
for i in A:
  df_sample = pd.DataFrame(columns = ['DMU', 'CRS_acc_mean', 'VRS_acc_mean', 'DNN_acc_mean','CRS_RMSE_mean','VRS_RMSE_mean','DNN_RMSE_mean'])
  df_sample['DMU'] = [i]
  df_sample['CRS_acc_mean'] = globals()["re_{}".format(i)]['CRS_acc'].mean()
  df_sample['VRS_acc_mean'] = globals()["re_{}".format(i)]['VRS_acc'].mean()
  df_sample['DNN_acc_mean'] = globals()["re_{}".format(i)]['DNN_acc'].mean()
  df_sample['CRS_RMSE_mean'] = globals()["re_{}".format(i)]['CRS_RMSE'].mean()
  df_sample['VRS_RMSE_mean'] = globals()["re_{}".format(i)]['VRS_RMSE'].mean()
  df_sample['DNN_RMSE_mean'] = globals()["re_{}".format(i)]['DNN_RMSE'].mean()

  df = pd.concat([df, df_sample], axis = 0)

df.reset_index(drop = True, inplace = True)
df_sample = pd.DataFrame(columns = ['DMU', 'CRS_acc_mean', 'VRS_acc_mean', 'DNN_acc_mean','CRS_RMSE_mean','VRS_RMSE_mean','DNN_RMSE_mean'])
df_sample['DMU'] = ['average']
df_sample['CRS_acc_mean'] = df['CRS_acc_mean'].mean()
df_sample['VRS_acc_mean'] = df['VRS_acc_mean'].mean()
df_sample['DNN_acc_mean'] = df['DNN_acc_mean'].mean()
df_sample['CRS_RMSE_mean'] = df['CRS_RMSE_mean'].mean()
df_sample['VRS_RMSE_mean'] = df['VRS_RMSE_mean'].mean()
df_sample['DNN_RMSE_mean'] = df['DNN_RMSE_mean'].mean()
df = pd.concat([df, df_sample], axis = 0)

In [ ]:
df

,DMU,CRS_acc_mean,VRS_acc_mean,DNN_acc_mean,CRS_RMSE_mean,VRS_RMSE_mean,DNN_RMSE_mean
0,10000,0.126460,0.250884,0.382361,0.346736,0.262549,0.162599
1,20000,0.129293,0.150274,0.391618,0.345194,0.338940,0.160813
2,40000,0.129576,0.153073,0.408601,0.340053,0.333477,0.161484
3,80000,0.132508,0.154843,0.418767,0.327840,0.323032,0.165425
4,160000,0.123932,0.178902,0.424152,0.339997,0.308097,0.164879
5,320000,0.119407,0.179300,0.428769,0.344694,0.301296,0.164612
6,640000,0.092616,0.253288,0.430754,0.381529,0.232987,0.167708
7,1280000,0.118547,0.177660,0.432256,0.353385,0.298823,0.166619
0,average,0.121542,0.187278,0.414660,0.347428,0.299900,0.164267


In [ ]:
df.to_csv('/content/gdrive/MyDrive/DNN_Frontier/0902_data/sample_result/1to128_feature_average.csv', encoding = 'UTF-8')

In [ ]:
total_df = pd.DataFrame(columns = ['DMU', 'Output', 'Input', 'Orientation', 'Card_Range', 'CRS_acc', 'VRS_acc', 'DNN_acc','acc_Applicable','CRS_RMSE','VRS_RMSE','DNN_RMSE','rmse_Applicable','Applicable'])
for i in A:
  total_df = pd.concat([total_df, globals()["re_{}".format(i)]], axis = 0)

In [ ]:
total_df

,DMU,Output,Input,Orientation,Card_Range,CRS_acc,VRS_acc,DNN_acc,acc_Applicable,CRS_RMSE,VRS_RMSE,DNN_RMSE,rmse_Applicable,Applicable
0,10000,1,3,1,0.10,0.068333,0.182222,0.368333,Y,0.371267,0.231816,0.152716,Y,Y
1,10000,1,3,1,0.05,0.033889,0.092778,0.180000,Y,0.372641,0.225918,0.157674,Y,Y
2,10000,1,3,1,0.20,0.176111,0.336111,0.605000,Y,0.369985,0.242441,0.162754,Y,Y
3,10000,1,3,2,0.10,0.037222,0.191111,0.225000,Y,0.459384,0.292755,0.264837,Y,Y
4,10000,1,3,2,0.05,0.013889,0.092222,0.096667,Y,0.463333,0.291905,0.251404,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,1280000,5,5,1,0.05,0.099761,0.136276,0.361931,Y,0.207072,0.174585,0.106239,Y,Y
128,1280000,5,5,1,0.20,0.370738,0.468911,0.730508,Y,0.220283,0.195768,0.116167,Y,Y
129,1280000,5,5,2,0.10,0.019245,0.427938,0.533867,Y,0.402759,0.129289,0.111519,Y,Y
130,1280000,5,5,2,0.05,0.008030,0.254905,0.360130,Y,0.401113,0.125332,0.113417,Y,Y


In [ ]:
total_df.to_csv('/content/gdrive/MyDrive/DNN_Frontier/0902_data/sample_result/1to128_result.csv', encoding = 'UTF-8')

In [ ]:
low_acc = total_df[total_df['DNN_acc'] <= 0.2]
low_acc

,DMU,Output,Input,Orientation,Card_Range,CRS_acc,VRS_acc,DNN_acc,acc_Applicable,CRS_RMSE,VRS_RMSE,DNN_RMSE,rmse_Applicable,Applicable
1,10000,1,3,1,0.05,0.033889,0.092778,0.180000,Y,0.372641,0.225918,0.157674,Y,Y
4,10000,1,3,2,0.05,0.013889,0.092222,0.096667,Y,0.463333,0.291905,0.251404,Y,Y
10,10000,1,4,2,0.05,0.022778,0.085000,0.100000,Y,0.422282,0.301464,0.264073,Y,Y
16,10000,1,5,2,0.05,0.041667,0.092222,0.105556,Y,0.385638,0.320813,0.260174,Y,Y
19,10000,2,2,1,0.05,0.027778,0.046111,0.156667,Y,0.349559,0.291819,0.202933,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,1280000,3,2,1,0.05,0.065117,0.065699,0.190608,Y,0.320536,0.304124,0.193369,Y,Y
73,1280000,4,1,1,0.05,0.066489,0.058064,0.121953,Y,0.297005,0.369375,0.265706,Y,Y
79,1280000,4,2,1,0.05,0.062096,0.059774,0.197808,Y,0.279132,0.290642,0.226131,Y,Y
103,1280000,5,1,1,0.05,0.066853,0.068177,0.122105,Y,0.295680,0.313839,0.301496,N,N


In [ ]:
low_vrs = total_df[total_df['VRS_acc'] <= total_df['CRS_acc']]
low_vrs

,DMU,Output,Input,Orientation,Card_Range,CRS_acc,VRS_acc,DNN_acc,acc_Applicable,CRS_RMSE,VRS_RMSE,DNN_RMSE,rmse_Applicable,Applicable
91,10000,4,4,1,0.05,0.105556,0.103333,0.221111,Y,0.255973,0.265453,0.129577,Y,Y
97,10000,4,5,1,0.05,0.118333,0.116667,0.265000,Y,0.276561,0.286165,0.114867,Y,Y
103,10000,5,1,1,0.05,0.017222,0.015000,0.096111,Y,0.454803,0.444285,0.214580,Y,Y
110,10000,5,2,1,0.20,0.250556,0.250000,0.510556,Y,0.303681,0.339738,0.209391,Y,Y
115,10000,5,3,1,0.05,0.088889,0.084444,0.190556,Y,0.264879,0.282980,0.144434,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,1280000,4,2,1,0.05,0.062096,0.059774,0.197808,Y,0.279132,0.290642,0.226131,Y,Y
84,1280000,4,3,1,0.10,0.127109,0.116797,0.425317,Y,0.321481,0.339296,0.157926,Y,Y
85,1280000,4,3,1,0.05,0.064679,0.057934,0.259002,Y,0.320568,0.340596,0.160113,Y,Y
108,1280000,5,2,1,0.10,0.125920,0.108173,0.344865,Y,0.328074,0.368596,0.215071,Y,Y


In [ ]:
unapplicable = total_df[total_df['Applicable'] == 'N']
unapplicable

,DMU,Output,Input,Orientation,Card_Range,CRS_acc,VRS_acc,DNN_acc,acc_Applicable,CRS_RMSE,VRS_RMSE,DNN_RMSE,rmse_Applicable,Applicable
4,20000,1,3,2,0.05,0.063056,0.051111,0.105833,Y,0.255707,0.368307,0.262952,N,N
10,20000,1,4,2,0.05,0.063333,0.055278,0.111944,Y,0.263983,0.372200,0.269686,N,N
43,40000,3,1,1,0.05,0.063889,0.065278,0.116944,Y,0.348471,0.278796,0.287645,N,N
16,80000,1,5,2,0.05,0.067569,0.049722,0.124861,Y,0.276648,0.384260,0.287935,N,N
42,80000,3,1,1,0.10,0.139444,0.096250,0.213333,Y,0.288767,0.398999,0.305605,N,N
43,80000,3,1,1,0.05,0.066250,0.070764,0.112500,Y,0.267744,0.311114,0.300990,N,N
102,80000,5,1,1,0.10,0.118264,0.128194,0.226736,Y,0.263627,0.265622,0.292521,N,N
104,80000,5,1,1,0.20,0.256528,0.267292,0.391319,Y,0.252982,0.328591,0.286225,N,N
16,160000,1,5,2,0.05,0.064687,0.057188,0.129236,Y,0.280422,0.346146,0.293756,N,N
43,160000,3,1,1,0.05,0.066389,0.069340,0.120208,Y,0.269332,0.330605,0.316020,N,N


In [ ]:
unapplicable.to_csv('/content/gdrive/MyDrive/DNN_Frontier/0902_data/sample_result/1to128_unapplicable.csv', encoding = 'UTF-8')

In [ ]:
worst_df = total_df[(total_df['VRS_acc'] <= total_df['CRS_acc']) & (total_df['Applicable'] == 'N')]
worst_df

,DMU,Output,Input,Orientation,Card_Range,CRS_acc,VRS_acc,DNN_acc,acc_Applicable,CRS_RMSE,VRS_RMSE,DNN_RMSE,rmse_Applicable,Applicable
4,20000,1,3,2,0.05,0.063056,0.051111,0.105833,Y,0.255707,0.368307,0.262952,N,N
10,20000,1,4,2,0.05,0.063333,0.055278,0.111944,Y,0.263983,0.372200,0.269686,N,N
16,80000,1,5,2,0.05,0.067569,0.049722,0.124861,Y,0.276648,0.384260,0.287935,N,N
42,80000,3,1,1,0.10,0.139444,0.096250,0.213333,Y,0.288767,0.398999,0.305605,N,N
16,160000,1,5,2,0.05,0.064687,0.057188,0.129236,Y,0.280422,0.346146,0.293756,N,N
44,320000,3,1,1,0.20,0.255833,0.111597,0.397326,Y,0.265309,0.462112,0.282287,N,N
72,320000,4,1,1,0.10,0.133698,0.131806,0.228750,Y,0.297399,0.293157,0.302785,N,N
72,640000,4,1,1,0.10,0.134453,0.132335,0.226997,Y,0.288742,0.285786,0.294683,N,N
73,640000,4,1,1,0.05,0.064028,0.060443,0.118837,Y,0.265312,0.273157,0.273584,N,N


In [10]:
# read test_categorized df
# read trad_eff_crs
# read trad_eff_vrs

# change their eff as card range 0.1 0.05
# calculate the accuracy
# compare with DNN

# extract the row which trad eff is better than DNN

# find out the reason why DNN Frontier is worse than traditonal Frontier some times
#######################################

# change trad eff as card range 0.2 ,0.1, 0.05
# compare with DNN

l = [10000] # DMU to
g = [20000] # DMU

h = [1,2,3,4,5] # Output
i = [1,2,3,4,5] # Input
j = [1,2] # Orientation
k = [1,2,3] # Card Range

#re = pd.DataFrame(columns = ['DMU', 'Output', 'Input', 'Orientation', 'Card_Range', 
#                             'CRS_acc', 'VRS_acc', 'DNN_acc','full_DNN_acc', 'acc_Applicable', 
#                             'CRS_RMSE', 'VRS_RMSE', 'DNN_RMSE', 'full_DNN_RMSE', 'rmse_Applicable', 
#                             'Applicable'])

re_df = pd.DataFrame(columns = ['DMU', 'Output', 'Input', 'Orientation', 'Card_Range', 
                             'CRS_acc', 'VRS_acc', 'DNN_acc', 'acc_Applicable', 
                             'CRS_RMSE', 'VRS_RMSE', 'DNN_RMSE', 'rmse_Applicable', 
                             'Applicable'])
m = 1 # 열 사이에 넣기 위한 도구

for f in l:
  re = pd.DataFrame(columns = ['2to{}_DNN_acc'.format(m), '2to{}_DNN_RMSE'.format(m)])
  # re = pd.concat([re_df,re], axis = 1)
  for a in g:
    for b in h:
      for c in i:
        for d in j:
          for e in k:
            
            if b + c < 4:
              continue


            r_1 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/comparison_result/result_comparison_{}.csv'.format(f) # comparison dataset
            r_2 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/trad_eff/{}_{}{}_{}_trad_eff.csv'.format(f,b,c,d) # CRS & VRS Result (not Categorized)
            r_3 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/re_DNN_Frontier_Classification/pred_df/other/Cat_{}to{}_{}{}_{}_{}_Pred_df.csv'.format(a,f,b,c,d,e) # Label Data & DNN Result (Categorized)

            df = pd.read_csv(r_3)
            df_2 = pd.read_csv(r_2)
            df = df.iloc[:,1:3]
            df_2 = df_2.iloc[:,1:3]

            # transform categorized_df
            if e == 1:
              bins = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
              lab = ['J','I','H','G','F','E','D','C','B','A']
            elif e == 2:
              bins = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
              lab = ['T','S','R','Q','P','O','N','M','L','K','J','I','H','G','F','E','D','C','B','A']
            else:
              bins = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
              lab = ['E','D','C','B','A']

            target_dat_1 = df_2.iloc[:,0]
            target_dat_2 = df_2.iloc[:,1]


            df_2['eff_lev_crs'] = pd.cut(target_dat_1, bins=bins, labels = lab)
            df_2['eff_lev_vrs'] = pd.cut(target_dat_2, bins=bins, labels = lab)        


            # target and transform trad_eff
            X = df_2.iloc[:, 2:4]

            LabE_1 = LabelEncoder()
            LabE_2 = LabelEncoder()
            
            Y_obj_1 = df_2['eff_lev_crs']
            Y_obj_2 = df_2['eff_lev_vrs']

            LabE_1.fit(Y_obj_1)
            Y_1 = LabE_1.transform(Y_obj_1)
            X['CRS_eff'] = Y_1

            LabE_2.fit(Y_obj_2)
            Y_2 = LabE_2.transform(Y_obj_2)
            X['VRS_eff'] = Y_2

            trad_eff_df = X.iloc[:,2:4]


            # transform label_range_to_singluar_data
            if e == 1:
              trad_eff_df['eff_crs_p'] = (2-(2*(trad_eff_df['CRS_eff'])+1)/10)/2
              trad_eff_df['eff_vrs_p'] = (2-(2*(trad_eff_df['VRS_eff'])+1)/10)/2
              
              df['eff_lev_p'] = (2-(2*(df['eff_lev'])+1)/10)/2
              df['pred_p'] = (2-(2*(df['pred'])+1)/10)/2

            elif e == 2:
              trad_eff_df['eff_crs_p'] = 1-(2*(trad_eff_df['CRS_eff'])+1)/40
              trad_eff_df['eff_vrs_p'] = 1-(2*(trad_eff_df['VRS_eff'])+1)/40

              df['eff_lev_p'] = 1-(2*(df['eff_lev'])+1)/40
              df['pred_p'] = 1-(2*(df['pred'])+1)/40

            else:
              trad_eff_df['eff_crs_p'] = 1-(2*(trad_eff_df['CRS_eff'])+1)/10
              trad_eff_df['eff_vrs_p'] = 1-(2*(trad_eff_df['VRS_eff'])+1)/10

              df['eff_lev_p'] = 1-(2*(df['eff_lev'])+1)/10
              df['pred_p'] = 1-(2*(df['pred'])+1)/10


  ################################################################################

            # calculate the accuracy
            df_dict = df.to_dict() # DNN Result
            dic_val = df_dict['eff_lev'].values()
            dict_list = list(dic_val)

            dnn_val = df_dict['pred'].values()
            dnn_list = list(dnn_val)


            trad_eff_df_dict = trad_eff_df.to_dict()
            dic_val_2 = trad_eff_df_dict['CRS_eff'].values()
            dict_list_2 = list(dic_val_2)

            dic_val_3 = trad_eff_df_dict['VRS_eff'].values()
            dict_list_3 = list(dic_val_3)

#            crs_acc = accuracy_score(dict_list, dict_list_2)
#            vrs_acc = accuracy_score(dict_list, dict_list_3)
            dnn_acc = accuracy_score(dict_list, dnn_list)

  ################################################################################

            # calculate RMSE
 #           crs_RMSE = mean_squared_error(df['eff_lev_p'], trad_eff_df['eff_crs_p'])**0.5
 #           vrs_RMSE = mean_squared_error(df['eff_lev_p'], trad_eff_df['eff_vrs_p'])**0.5
            dnn_RMSE = mean_squared_error(df['eff_lev_p'], df['pred_p'])**0.5

  ################################################################################

            ## Result
            n_re = pd.DataFrame(columns = ['2to{}_DNN_acc'.format(m), '2to{}_DNN_RMSE'.format(m)])
            n_re['2to{}_DNN_acc'.format(m)] = [dnn_acc]
            n_re['2to{}_DNN_RMSE'.format(m)] = [dnn_RMSE]

            re = pd.concat([re, n_re], axis = 0)

            re_df = pd.read_csv(r_1)
  m = m*2
  re.reset_index(drop = True, inplace = True)
  re.to_csv('/content/gdrive/MyDrive/DNN_Frontier/0902_data/comparison_result/comparison_20000to{}.csv'.format(f), encoding = 'UTF-8')